In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from mystochastic import generate_stochastic_scenarios, evaluate_expected_profit

# --- System and Financial Constants ---
BESS_CAPACITY = 50  # MWh
RT_AFRR_PRICE = 250 # €/MWh for aFRR service delivery
HOURS = 24

ModuleNotFoundError: No module named 'mystochastic'

In [ ]:
# --- 1. Base Price Forecast (Using simple regression for example) ---

def get_base_forecast(hours):
    """Generates a simple base price forecast curve."""
    # Synthetic data similar to previous steps
    X_hist = np.array([5, 15, 25, 35, 45, 55]).reshape((-1, 1))
    y_hist = np.array([50, 75, 60, 90, 80, 110])
    model = LinearRegression().fit(X_hist, y_hist)
    
    X_forecast = np.arange(60, 60 + hours * 10, 10).reshape((-1, 1))
    base_prices = model.predict(X_forecast)
    return np.maximum(5, base_prices)

base_prices = get_base_forecast(HOURS)

In [3]:
# --- 2. Stochastic Optimization (Stage 1 Decision) ---

# Define the capacity steps to test for RT reservation (e.g., 0% to 100% of capacity)
RESERVATION_STEPS = np.linspace(0, BESS_CAPACITY, 11) # Test 0, 5, 10, ..., 50 MWh

# Generate price scenarios
NUM_SCENARIOS = 10
VOLATILITY_FACTOR = 0.20 # 20% volatility for RT price

scenarios_df = generate_stochastic_scenarios(
    base_prices, num_scenarios=NUM_SCENARIOS, vol_factor=VOLATILITY_FACTOR
)

# Run the Stochastic Optimization (Iterate through reservation options)
optimization_results = []
for capacity_reserved in RESERVATION_STEPS:
    
    # Calculate expected profit for this reservation level
    expected_profit, scenario_profits_df = evaluate_expected_profit(scenarios_df, capacity_reserved)
    
    optimization_results.append({
        'Reserved_Capacity_MWh': capacity_reserved,
        'Expected_Profit': expected_profit,
        'Std_Dev_Profit': scenario_profits_df['Total_Profit'].std()
    })

optimization_df = pd.DataFrame(optimization_results)

# --- Find the Optimal Solution ---
optimal_row = optimization_df.loc[optimization_df['Expected_Profit'].idxmax()]
optimal_capacity_reserved = optimal_row['Reserved_Capacity_MWh']
optimal_expected_profit = optimal_row['Expected_Profit']

print("--- Stochastic Optimization Results ---")
print(f"BESS Capacity: {BESS_CAPACITY} MWh | RT (aFRR) Price: {RT_AFRR_PRICE} €/MWh")
print(f"Optimal Reserved Capacity (aFRR): {optimal_capacity_reserved:.2f} MWh")
print(f"Maximum Expected Daily Profit: {optimal_expected_profit:.2f} €")
print("-" * 50)

NameError: name 'BESS_CAPACITY' is not defined

In [2]:
# --- Plot 1: Probabilistic Price Scenarios ---
plt.figure(figsize=(12, 6))
scenarios_df.filter(like='RT_Price').iloc[:, :5].plot(
    legend=False, alpha=0.6, linestyle='--', color='gray'
)
plt.plot(base_prices, label='Base DA Forecast', color='blue', linewidth=3)
plt.title(f'Probabilistic Real-Time Price Scenarios (N={NUM_SCENARIOS})')
plt.xlabel('Hour of Day')
plt.ylabel('Price (€/MWh)')
plt.legend()
plt.grid(True)
plt.savefig('stochastic_scenarios.png')

# --- Plot 2: Expected Profit vs. Reserved Capacity (Stage 1 Decision) ---
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot Expected Profit
ax1.plot(optimization_df['Reserved_Capacity_MWh'], 
         optimization_df['Expected_Profit'], 
         marker='o', color='green', label='Expected Profit (€)')
ax1.axvline(optimal_capacity_reserved, color='red', linestyle='--', 
            label=f'Optimal Reserve ({optimal_capacity_reserved:.0f} MWh)')
ax1.set_xlabel('Capacity Reserved for RT (aFRR) [MWh]')
ax1.set_ylabel('Expected Profit (€)', color='green')
ax1.tick_params(axis='y', labelcolor='green')
ax1.set_title('Stage 1 Decision: Expected Profit vs. Capacity Reservation')
ax1.legend(loc='upper left')

# Plot Profit Risk (Standard Deviation)
ax2 = ax1.twinx() 
ax2.plot(optimization_df['Reserved_Capacity_MWh'], 
         optimization_df['Std_Dev_Profit'], 
         marker='x', color='blue', linestyle=':', label='Risk (Std Dev of Profit)')
ax2.set_ylabel('Profit Risk (Std Dev) [€]', color='blue')
ax2.tick_params(axis='y', labelcolor='blue')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.savefig('stochastic_optimization_result.png')

print("\nAll stochastic optimization steps and graphs generated.")
print("Generated files: bess_stochastic_helpers.py (functions), bess_stochastic_main.py (main script)")
print("Generated plots: stochastic_scenarios.png, stochastic_optimization_result.png")

NameError: name 'scenarios_df' is not defined

<Figure size 1200x600 with 0 Axes>